In [ ]:
from TempMail import TempMail
# Create a new TempMail object
tmp = TempMail()
# Generate an inbox
inb = TempMail.generateInbox(tmp)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from TempMail import TempMail
import time
import requests
from elevenlabs import set_api_key
import os
import shutil
import random
import subprocess
from pydub import AudioSegment
from dataclasses import dataclass



@dataclass
class ElevenMetadata:
    url: str
    character: str


voices = {

'Player' : ElevenMetadata('GTV0x4o0Qi6q9wOEtj0j','Bezi'), #gothic 
'Weasel' : ElevenMetadata('6gzVd5bQRdJEFSpl9wFZ', 'Lester'), #gothic
'Doug' : ElevenMetadata('FW2cbJ9jNc5W85HgH4Rk', 'Milten'), #gothic
'Belschwur' : ElevenMetadata('ZJNhgFtk5TFUd1vlbOTj', 'Gorn'), #gothic 
'Sam' : ElevenMetadata('NbmNvjFyX47rxmLJUQYe', 'Diego'), #gothic
'Druid' : ElevenMetadata('RNraAeOnpQYSMr1S0XMk', 'Thorus'), #gothic
'Domingo' : ElevenMetadata('snADbfDYI3cUD8D9CCkv', 'Cronos'), #gothic
'Abrax' : ElevenMetadata('zrKCscmYoLagE5zxEQBn','CorAngar'), #gothic
'Inquisitor' : ElevenMetadata('o1tDFSX9b7lxHQdrZb9Q','Saturas'), #gothic
'Santiago' : ElevenMetadata('RdDxchtUFBhZ83Xpu4l8','YBerion'), #gothic

'Don' : ElevenMetadata('430s98E5c0sIiJyD4hba','bando'), #actor
'Ash' : ElevenMetadata('HxfDPZwKIjgNobkiXnUm','bobrowski'), #actor    
'Leto' : ElevenMetadata('N4h6Ruad67n3bL9YyyPe','kosior'), #actor
'Luis' : ElevenMetadata('19bryV7gtGpWewJT0q0K','bajtlik'), #actor
'Brogar' : ElevenMetadata('XVIUyqPWx5F27W5Cj9sL','jarocinski'),
'Lorenzo' : ElevenMetadata('eBLVrchg51jUUbTFNctY','lukomski'),
'Delgado' : ElevenMetadata('23LX9oSMcKHCblT5PKiW','grab_ab'),
'Fincher' : ElevenMetadata('OcA1hr3lKP0jTTch3W1O','zebrowski'), #actor
'Hernandez' : ElevenMetadata('2eVyhiLEJG1uo1G3dEF0','corso'),

'Romanov' : ElevenMetadata('WzfRzEUKBZTnIqkuKzo0','fraczewski'), #actor
'Pallas' : ElevenMetadata('JXRWN4SYDM6cO7Zd3dzL','grabowski'), #actor

'Danny' : ElevenMetadata('XRFbAlstsKmR5foqbpkw','gajos'), #actor
    
'Enzo' : ElevenMetadata('8kfxereQJPj7mvAGLS0j','Bufford'),
'Carlos' : ElevenMetadata('RSypxcY25h5wnC5RDZL3','defoe'),
'Sabrosa' : ElevenMetadata('dEwWyanp50wFer63vADR','Zuben'),
'Olf' : ElevenMetadata('rMCYAYHGKryyhnOYkLO5','Gonzales'),
'Ogre' : ElevenMetadata('6cv9sJrwrTsEB5HNgMai','Gorn_g3'),
}


def get_countries():
    with open('/home/jakubg/countries_out', 'r') as file:
        lines = file.readlines()
    edited_countries = [ str(country.strip()) for country in lines]
    return edited_countries

def run_vpn():
    edited_countries = get_countries()    
    selected_country = random.choice(edited_countries)
    command = f'nordvpn connect {selected_country}'
    result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

def get_driver_settings():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    driver = webdriver.Chrome(options=chrome_options)
    return driver
        

def automate(output_directory,eleven_directory):
    run_vpn()
    driver = get_driver_settings()
    sign_up_and_verify_email(driver)
    key = get_api_key(driver)
    time.sleep(4)   
    eleven(eleven_directory, output_directory,api_key)
    time.sleep(4)
    driver.quit()
    !nordvpn disconnect 
    
    
def eleven(path,out,api_key):

    os.makedirs(out, exist_ok=True)
    for directory in os.listdir(path):
        dir_path = os.path.join(path, directory)
        os.makedirs(os.path.join(out,directory),exist_ok=True)
        
        metadata = get_voice(voices[directory],api_key)                                      #struktura plikow GLOWNY_FOLDER => CHAPTER_X =>  POSTAC_NAZWA -> PLIK_X_POSTAC_NAZWA._xsnd
        
        for object in os.listdir(dir_path):                                       
            object_path = os.path.join(dir_path,object)
            os.makedirs(os.path.join(out,directory,object),exist_ok=True)                    # object == klucz
            metadata = get_voice(voices[object],api_key)
            
            for files in os.listdir(object_path):
                file_path = os.path.join(object_path, files)
                out_path = os.path.join(out,directory,object,files) + ".mp3"
                if not os.path.exists(out_path):
                    
                    with open(file_path,'r') as f:
                        text = f.read()
                        print(out_path)
                            
                            if not os.path.exists(out_path):
                                url = f"https://api.elevenlabs.io/v1/text-to-speech/{metadata.url}"
                                response = requests.post(url, json=data_eleven, headers=headers)
                                with open(f'{out_path}', 'wb') as f:
                                    for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
                                        if chunk:
                                            f.write(chunk)     


def get_api_key(driver):
    button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'button[data-testid="user-menu-button"]'))
    )

    button.click()

    button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'headlessui-menu-item-P0-30'))
    )
    button.click()
    
    time.sleep(30)
    show_key_button = driver.find_element(By.CSS_SELECTOR, "#headlessui-dialog-panel-P0-0 > div > div.mt-3.w-full > div.space-y-3 > div:nth-child(2) > div > button.flex.items-center.btn.btn-white.btn-md.btn-normal.border-l-0.rounded-none")
    show_key_button.click()
    time.sleep(30)
    input_field = driver.find_element(By.CSS_SELECTOR, '#headlessui-dialog-panel-P0-0 > div > div.mt-3.w-full > div.space-y-3 > div:nth-child(2) > div > div > input')

    key = input_field.get_attribute("value")
    time.sleep(2)
    return key


def sign_up_and_verify_email(driver):
    tmp = TempMail()
    inb = TempMail.generateInbox(tmp)

    address = inb.address
    part = address.split('@')
    passwd = part[0]

    driver.get('https://elevenlabs.io/sign-up')

    email_input = driver.find_element(By.CSS_SELECTOR, 'input[data-testid="signup-email-input"]')
    email_input.send_keys(address)
    password_input = driver.find_element(By.CSS_SELECTOR, 'input[type="password"][autocomplete="current-password"]')
    password_input.send_keys(passwd)

    signup_button = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
    signup_button.click()

    time.sleep(20)
    emails = TempMail.getEmails(tmp, inbox=inb)
    first_email = emails[0]
    email_content = first_email.body

    with open("file.txt","w") as file:
        file.write(email_content)
    
    with open("file.txt","r") as file:
        for line in file:
            if "Verify Email" in line:
                link = line[15:-2]
    driver.get(link)

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "headlessui-dialog-panel-P0-0")))

    close_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Close')]")
    close_button.click()

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="email"]')))

    email_login = driver.find_element(By.CSS_SELECTOR, 'input[type="email"]')
    email_login.send_keys(address)
    passwd_login = driver.find_element(By.CSS_SELECTOR, 'input[type="password"]')
    passwd_login.send_keys(passwd)
    button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
    button.click()

def get_voice(data,key):
    url = f"https://api.elevenlabs.io/v1/voices/{data}?with_settings=true"

    headers = {
        "accept": "application/json",
        "xi-api-key": f"{key}"
    }

    response = requests.get(url, headers=headers)







    

In [ ]:
if __name__ == "__main__":
    eleven_directory = '/home/jakubg/Desktop/infered' 
    output_directory = '/home/jakubg/Desktop/test'
     
    while True:
        automate(output_directory,eleven_directory)